
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/floydluo/Library/CloudStorage/OneDrive-DRFIRST.COM,INC/Documents - DrFirst-JHU Collaboration Center/DrFirst-Project/2024-DrFirst-v2-SPACE/_DrFirst-AI-EduRxPred-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_EduRxPred',
 'MODEL_TASK': 'Model_EduRxPred',
 'WORKSPACE_PATH': '/Users/floydluo/Library/CloudStorage/OneDrive-DRFIRST.COM,INC/Documents '
                   '- DrFirst-JHU Collaboration '
                   'Center/DrFirst-Project/2024-DrFirst-v2-SPACE/_DrFirst-AI-EduRxPred-WorkSpace'}


# Trigger Events

In [2]:
###########################
TriggerRecName = 'Rx'
case_id_columns = ["PID", "ObsDT", 'PInvID', 'RxID']
special_columns = ['PID', 'DT', 'PInvID', 'RxID', 'show_educational_campaigns', 'drug_name', 'ndc_id']
###########################

In [3]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args

cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
print(cohort_args)

{'RawRootID': 'patient_id_encoded', 'CohortInfo': {'RawData2023Nov_Trulicity': {'cohort_label': 1, 'cohort_name': 'RawData2023Nov_Trulicity', 'FolderPath': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity'}}, 'RootID': 'PID', 'RecName': 'PRawRecNum', 'rec_folder': '../_Data/1-Data_RFT/RecAttr/', 'fld_folder': '../_Data/1-Data_RFT/RecAttr/', 'hfds_folder': '../_Data/1-Data_RFT/HfDataset/', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'RecName_to_RFT_GROUP_SIZE': {'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'Default': 10000}, 'RecName_to_RFT_usebucket': {'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_path': '../pipeline/config_recfldtkn/', 'SPACE': {'DATA_RAW': '../_Data/0-Data_Raw', 'DATA_RFT': '../_Data/1-Data_RFT', 'DATA_CaseObs': '../_Data/2-Data_CaseObs', 'CODE_FN': '../pipeline', 'CODE_RFT': '../pipeline', 'DATA_TASK': 'Data_EduRxPred', 'MODEL_TASK': 'Model_EduRxPred', '

In [4]:
CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder')
print(CaseFolder)

Data_EduRxPred/CaseFolder


# ScopeCaseName: TrulicityRx

In [5]:
######################################
ScopeCaseName = 'TrulicityRx'

def convert_trigger_events_to_caseset(TriggerRecName, case_id_columns, special_columns, cohort_args):
    # record_args = load_record_args(TriggerRecName, cohort_args)
    ds_rec, _ = load_ds_rec_and_info(TriggerRecName, cohort_args)
    # ds_rec = ds_rec.rename_column(record_args['RecDT'], 'ObsDT')
    # print(ds_rec.head())
    ds_case = ds_rec.select_columns(special_columns)
    df_case_raw = ds_case.to_pandas()

    # ------------------------------ Filter Cases ------------------------------
    df_case_filter = df_case_raw[df_case_raw['drug_name'] == 'Trulicity'].reset_index(drop=True)
    # --------------------------------------------------------------------------

    # ------------------------------ Determine the ObsDT ------------------------------
    df_case_filter['ObsDT'] = df_case_filter['DT'] # [df_case_raw['drug_name'] == 'Trulicity']
    # --------------------------------------------------------------------------

    # ------------------------------- Update Column Sequence ------------------------
    columns = df_case_filter.columns 
    columns = case_id_columns + [col for col in columns if col not in case_id_columns]
    df_case_filter = df_case_filter[columns].reset_index(drop=True)

    return df_case_filter
######################################


df_case_filter = convert_trigger_events_to_caseset(TriggerRecName, case_id_columns, special_columns, cohort_args)
df_case_filter

,PID,ObsDT,PInvID,RxID,DT,show_educational_campaigns,drug_name,ndc_id
0,1000001,2023-09-07 18:13:08.746,1000001-005,1000001-005-000,2023-09-07 18:13:08.746,True,Trulicity,2143380.0
1,1000002,2023-09-21 23:38:11.348,1000002-000,1000002-000-000,2023-09-21 23:38:11.348,True,Trulicity,2143380.0
2,1000003,2023-09-05 16:33:52.699,1000003-000,1000003-000-000,2023-09-05 16:33:52.699,True,Trulicity,2143380.0
3,1000004,2023-05-09 20:46:13.911,1000004-003,1000004-003-000,2023-05-09 20:46:13.911,None,Trulicity,2143380.0
4,1000004,2023-09-01 17:07:57.276,1000004-007,1000004-007-000,2023-09-01 17:07:57.276,True,Trulicity,2223680.0
...,...,...,...,...,...,...,...,...
101001,1046209,2023-05-04 13:45:40.707,1046209-000,1046209-000-000,2023-05-04 13:45:40.707,None,Trulicity,2143380.0
101002,1046209,2023-07-18 15:48:38.928,1046209-002,1046209-002-000,2023-07-18 15:48:38.928,True,Trulicity,2143380.0
101003,1046209,2023-07-20 15:05:52.683,1046209-004,1046209-004-000,2023-07-20 15:05:52.683,True,Trulicity,2143380.0
101004,1046209,2023-09-07 18:10:13.007,1046209-005,1046209-005-000,2023-09-07 18:10:13.007,True,Trulicity,2143380.0


In [6]:
df_case = df_case_filter

# ScopeCaseName: EduCmpRx

In [7]:
# ######################################
# ScopeCaseName = 'EduCmpRx'

# def convert_trigger_events_to_caseset(TriggerRecName, case_id_columns, special_columns, cohort_args):
#     record_args = load_record_args(TriggerRecName, cohort_args)
#     ds_rec, _ = load_ds_rec_and_info(TriggerRecName, cohort_args)
#     ds_case = ds_rec.select_columns(special_columns)
#     df_case_raw = ds_case.to_pandas()

#     # ------------------------------ Filter Cases ------------------------------
#     df_case_filter = df_case_raw[df_case_raw['show_educational_campaigns'] == True].reset_index(drop=True)
#     # --------------------------------------------------------------------------

#     # ------------------------------ Determine the ObsDT ------------------------------
#     df_case_filter['ObsDT'] = df_case_filter['DT'] 
#     # --------------------------------------------------------------------------

#     # ------------------------------- Update Column Sequence ------------------------
#     columns = df_case_filter.columns 
#     columns = case_id_columns + [col for col in columns if col not in case_id_columns]
#     df_case_filter = df_case_filter[columns].reset_index(drop=True)
    
#     return df_case_filter
# ######################################

# df_case_filter = convert_trigger_events_to_caseset(TriggerRecName, case_id_columns, special_columns, cohort_args)
# df_case_filter

# Case SubGroup

In [8]:
# from recfldtkn.datapoint import load_df_data_with_RecName
#########################
subgroup_columns = ['patient_gender', 'patient_age_bucket', 'patient_zipcode_3']
#########################

ds_P, _ = load_ds_rec_and_info('P', cohort_args)
RootID = 'PID'
ds_P = ds_P.select_columns([RootID] + subgroup_columns)
df_P = ds_P.to_pandas()

df_P['patient_gender'] = df_P['patient_gender'].apply(lambda x: 'g' + str(x) if str(x) != 'U' else 'gNone')
# df_P['patient_age_bucket'] = df_P['patient_age_bucket'].apply(lambda x: 'a' + str(x).replace('+', '-').split('-')[0])
df_P['cohort'] = df_P[RootID].apply(lambda x: 'C' + str(x)[0])
df_P.head()

,PID,patient_gender,patient_age_bucket,patient_zipcode_3,cohort
0,1000001,gM,41-50,494.0,C1
1,1000002,gF,51-60,350.0,C1
2,1000003,gF,71-80,213.0,C1
3,1000004,gF,71-80,497.0,C1
4,1000005,gF,31-40,859.0,C1


In [9]:
df_P['patient_gender'].value_counts()

patient_gender
gF       26122
gM       19615
gNone       54
Name: count, dtype: int64

In [10]:
def get_subgroup_label_of_age(x):
    x = str(x).split('-')[0]
    if x == 'None':
        return 'None' 
    elif x in ['51', '61', '71', '81+']:
        return 'Ab50'
    else:
        return 'Bl50'

df_P['patient_age_group'] = df_P['patient_age_bucket'].apply(lambda x: get_subgroup_label_of_age(x))
df_P['patient_age_group'].value_counts()

patient_age_group
Ab50    33360
Bl50    12398
None       33
Name: count, dtype: int64

In [11]:
df_case = pd.merge(df_case, df_P[[RootID, 'cohort', 'patient_gender', 'patient_age_group', 'patient_age_bucket', 'patient_zipcode_3']], how = 'left')
df_case.head()

,PID,ObsDT,PInvID,RxID,DT,show_educational_campaigns,drug_name,ndc_id,cohort,patient_gender,patient_age_group,patient_age_bucket,patient_zipcode_3
0,1000001,2023-09-07 18:13:08.746,1000001-005,1000001-005-000,2023-09-07 18:13:08.746,True,Trulicity,2143380.0,C1,gM,Bl50,41-50,494.0
1,1000002,2023-09-21 23:38:11.348,1000002-000,1000002-000-000,2023-09-21 23:38:11.348,True,Trulicity,2143380.0,C1,gF,Ab50,51-60,350.0
2,1000003,2023-09-05 16:33:52.699,1000003-000,1000003-000-000,2023-09-05 16:33:52.699,True,Trulicity,2143380.0,C1,gF,Ab50,71-80,213.0
3,1000004,2023-05-09 20:46:13.911,1000004-003,1000004-003-000,2023-05-09 20:46:13.911,None,Trulicity,2143380.0,C1,gF,Ab50,71-80,497.0
4,1000004,2023-09-01 17:07:57.276,1000004-007,1000004-007-000,2023-09-01 17:07:57.276,True,Trulicity,2223680.0,C1,gF,Ab50,71-80,497.0


In [12]:
df_case['patient_gender'].value_counts()

patient_gender
gF       58193
gM       42756
gNone       57
Name: count, dtype: int64

# [Part 2] Two Ways to Seperate Cases

1. Case Subgroup Type: 
2. Case Split Type: Downsample-In/Out-Train/Validation/Test

# [Step 1] Subgroups: by Age / Gender / Zipcode / Drug Name

In [13]:
df_case.shape

(101006, 13)

In [14]:
##############################
df_case['group'] = df_case['cohort'] + '-'  + df_case['patient_gender'] + '-' + df_case['patient_age_group']
df_case['group'] = df_case['group'].apply(lambda x: x if 'None' not in x else 'None')
##############################

df_case['group'].value_counts().sort_index()

group
C1-gF-Ab50    40517
C1-gF-Bl50    17676
C1-gM-Ab50    32213
C1-gM-Bl50    10543
None             57
Name: count, dtype: int64

# ObsDT Distribution

In [15]:
df_case['ObsDT'].min(), df_case['ObsDT'].max()

(Timestamp('2023-03-06 01:14:34.993000'),
 Timestamp('2023-11-30 18:51:55.732000'))

In [16]:
for d in ['2023-11-01', '2023-10-01']:
    s = df_case['ObsDT'] >= pd.to_datetime(d)
    print(d, s.mean())

2023-11-01 0.06178840860938954
2023-10-01 0.13078430984297962


# Save to SubGroups

In [18]:
group_name_list = sorted(list(df_case['group'].unique())) 
group_name_to_id = {name: i for i, name in enumerate(group_name_list)}
group_name_to_id

{'C1-gF-Ab50': 0, 'C1-gF-Bl50': 1, 'C1-gM-Ab50': 2, 'C1-gM-Bl50': 3, 'None': 4}

In [19]:
case_type = ScopeCaseName

for group_name, df_case_group in df_case.groupby('group'):
    # df_case_group = df_case_group[case_id_columns].reset_index(drop = True)
    df_case_group = df_case_group.reset_index(drop = True)
    id = group_name_to_id[group_name]
    path = os.path.join(CaseFolder, case_type, f'{id}_{group_name}' + '.p')
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    print(path, len(df_case_group))
    df_case_group.to_pickle(path)


Data_EduRxPred/CaseFolder/TrulicityRx/0_C1-gF-Ab50.p 40517
Data_EduRxPred/CaseFolder/TrulicityRx/1_C1-gF-Bl50.p 17676
Data_EduRxPred/CaseFolder/TrulicityRx/2_C1-gM-Ab50.p 32213
Data_EduRxPred/CaseFolder/TrulicityRx/3_C1-gM-Bl50.p 10543
Data_EduRxPred/CaseFolder/TrulicityRx/4_None.p 57


In [20]:
df_case_group.head()

,PID,ObsDT,PInvID,RxID,DT,show_educational_campaigns,drug_name,ndc_id,cohort,patient_gender,patient_age_group,patient_age_bucket,patient_zipcode_3,group
0,1000564,2023-07-28 22:00:54.781,1000564-002,1000564-002-000,2023-07-28 22:00:54.781,True,Trulicity,2318201.0,C1,gNone,Ab50,71-80,985.0,None
1,1000564,2023-09-14 15:03:22.234,1000564-003,1000564-003-000,2023-09-14 15:03:22.234,True,Trulicity,2318280.0,C1,gNone,Ab50,71-80,985.0,None
2,1000597,2023-03-14 19:47:03.422,1000597-000,1000597-000-000,2023-03-14 19:47:03.422,None,Trulicity,2223601.0,C1,gNone,Ab50,61-70,985.0,None
3,1000597,2023-04-06 23:56:35.534,1000597-001,1000597-001-000,2023-04-06 23:56:35.534,None,Trulicity,2318201.0,C1,gNone,Ab50,61-70,985.0,None
4,1000597,2023-05-03 20:54:56.970,1000597-002,1000597-002-000,2023-05-03 20:54:56.970,None,Trulicity,2318201.0,C1,gNone,Ab50,61-70,985.0,None
